In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
# np.set_printoptions(threshold=sys.maxsize)

In [2]:
Ne = 800
Ni =200
re = np.random.random((Ne,1))
ri = np.random.random((Ni,1))
a = np.concatenate([0.02*np.ones((Ne,1)), 0.02+0.08*ri])
b = np.concatenate([0.2*np.ones((Ne,1)), 0.25 - 0.05*ri])
c = np.concatenate([-65 + 15*re**2, -65*np.ones((Ni,1))])
d = np.concatenate([8 - 6*re**2, 2*np.ones((Ni,1))])
S =  np.hstack([0.5*np.random.random((Ne+Ni, Ne)), -np.random.random((Ne+Ni, Ni))])


In [3]:
v= -65*np.ones((Ne+Ni,1)) # Initial values of v
u=b*v                     # initial values of u
firings = np.empty((0, 2))
blocks = []

In [4]:
I_mem = []
v_mem = []
v_mem_ = []
for t in np.arange(1000):
    I = np.vstack([5*np.random.randn(Ne,1), 2*np.random.randn(Ni,1)])
    # fired= np.where(v>=30)[0]
    fired = np.flatnonzero(v >= 30)   # (m,)
    if fired.size:
        v[fired] = 30
        
        v_mem.append(v.copy())
        
        blocks.append(np.column_stack((np.full(fired.size, t, dtype=int), fired)))
        v[fired] = c[fired]
        u[fired] = u[fired]+d[fired]
        I = I+np.sum(S[:, fired], axis=1, keepdims=True)
    else:
        v_mem.append(v.copy())
        
        # break
    v = v+0.5*(0.04*v**2 + 5*v + 140 - u + I)                                      # step 0.5 ms
    v = v+0.5*(0.04*v**2 + 5*v + 140 - u + I)                                      # for numerical stability
    u = u+a * (b*v-u)

    I_mem.append(I.copy())

    # break
firings = np.concatenate(blocks, axis=0) if blocks else np.empty((0, 2), dtype=int)
I_mem = np.array(I_mem)
v_mem = np.array(v_mem)
v_mem.shape

(1000, 1000, 1)

In [14]:
i=2
offset=100
%matplotlib qt
plt.figure(figsize=(12,6))
plt.scatter(*firings[firings.T[1] <800].T,s=1, marker='|')
plt.scatter(*firings[firings.T[1] >=800].T,s=1, c='firebrick', marker='|')
plt.plot(v_mem[:,i],color='black')
plt.axhline(800, color='black',alpha=0.5)
plt.title('1000 randomly coupled spiking neurons')
plt.xlabel('time, ms')
plt.ylabel('neuron number')
plt.show()